# T9 - Artificial intelligence (T-DAT-901)
## Travel Order Resolver
Project members:
* Matisse AUBRY
* Georges BITAKWIRE
* Loan JOUFFROY
* Raphaël LÉVY 
* Marco VIALLEFONT

## Project setup

In [44]:
!pip install langdetect
!python -m spacy download fr_core_news_sm
!apt remove libav-tools
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install geograpy3==0.1.2
!pip install PyAudio
!pip install ffmpeg-python
!pip install SpeechRecognition
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read, wav_open
import io
import ffmpeg
from langdetect import detect
import fr_core_news_sm
from nltk.corpus import stopwords
import nltk
import geograpy
import speech_recognition as sr
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')


nlp = fr_core_news_sm.load()
print("Project setup")

     |████████████████████████████████| 14.7 MB 4.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package 'libav-tools' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1).
libportaudiocpp0 is already the newest version (19.6.0-1).
portaudio19-dev is already the newest version (19.6.0-1).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...

In [123]:
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  audio = wav_read(io.BytesIO(riff))

  return audio

In [128]:
audio = get_audio()
print(audio)
type(audio)

b'RIFFFI\x02\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\x80\xbb\x00\x00\x00w\x01\x00\x02\x00\x10\x00LIST\x1a\x00\x00\x00INFOISFT\x0e\x00\x00\x00Lavf57.83.100\x00data\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

bytes

In [127]:
import speech_recognition as sr
r = sr.Recognizer()

audio_source = sr.AudioData(audio,16000, 2)
print(audio_source)

data = r.recognize_google(audio_source, language="fr-FR", show_all = True)
print(data)

[]


speech_recognition.AudioData

In [ ]:
LANGUAGES_ACCEPTED = ['fr'] # => If, one day, we wish add new languages

# Return True if the input have a accpected language, else return False
def check_if_text_accepted_language(text):
    accepted = False
    lang_detected = detect(text)
    for langue in LANGUAGES_ACCEPTED:
        if (langue == lang_detected):
            accepted = True
    return accepted

In [ ]:
def return_arrival(sentence):
  if(sentence == ""):
    print("veuillez saisir votre phrase")
  else:
    doc = nlp(sentence)
    verb = [token.lemma_ for token in doc]
    phrase_sent = [(X, X.pos_) for X in doc]
    places = geograpy.get_place_context(text=sentence)
  #     nlp_wk = spacy.load('xx_ent_wiki_sm')
      
    adp =  [word for word,pos in phrase_sent if pos == 'ADP']
    prop = [word for word,pos in phrase_sent if pos == 'PROPN']
    city = [(X.text) for X in doc.ents]
    # print(adp)
    if(len(adp)>1):
      adp = str(adp)
      # print(city)
      adp = adp.replace("pour,", "")
      adp = adp.replace("Bonjour,", "")
      adp = adp.replace("en,", "")
      adp = adp.replace("en", "")
      adp = adp.replace("dans,", "")
      adp = adp.replace(" ", "")
      
      # print(adp)
      adp = nlp(adp)
      
      adp_word =  [(X, X.pos_) for X in adp]
      # print(adp_word)
      real_adp = [word for word,pos in adp_word if pos == 'ADP'] 
      adp = real_adp
      # print(len(str(adp)))
    dictionary = {"departure":["de","depuis","à", "par","vers"]} 
    for cle, valeur in dictionary.items():
      if not adp or valeur[0] == str(adp[0]):
        if len(city) == 1:
          city = str(city)
          city = nlp(city)
          city_word = [(X, X.pos_) for X in city]
          real_city = [word for word,pos in city_word if pos == 'PROPN'] 
          city = real_city
        if(str(city[1]) == ""):
          print("arrival : ",prop[1])
        else:
          print("arrival : ",city[1])
      elif valeur[4] == str(adp[0]):
        if(str(city[1]) == ""):
          print("arrival : ",prop[1])
        else:
          print("arrival : ",city[1])
      elif valeur[2] == str(adp[0]) and str(verb[1]) == "être":
        if(str(city[1]) == ""):
          print("arrival : ",prop[1])
        else:
          print("arrival : ",city[1])
      elif valeur[2] == str(adp[0]):
        if(str(city[0]) == ""):
          print("arrival : ",prop[0])
        else:
          print("arrival : ",city[0])
      elif valeur[2] == str(adp[0]):
        if(str(city[0]) == ""):
          print("arrival : ",prop[0])
        else:
          print("arrival : ",city[0])
      elif valeur[1] == str(adp[0]) or str(adp[1]):
        if(str(city[0]) == ""):
          print("arrival : ",prop[0])
        else:
          print("arrival : ",city[0])
      elif valeur[3] == str(adp[1]):
        if(str(city[0]) == ""):
          print("arrival : ",prop[0])
        else:
          print("arrival : ",city[0])
      else:
        if(str(city[1]) == ""):
          print("arrival : ",prop[1])
        else:
          print("arrival : ",city[1])
return_arrival("Je veux aller de Marseille vers Paris")

arrival :  Paris


In [ ]:
def return_departure(sentence):
  if(sentence == ""):
    print("veuillez saisir votre phrase")
  else:
    doc = nlp(sentence)
    verb = [token.lemma_ for token in doc]
    phrase_sent = [(X, X.pos_) for X in doc]
    places = geograpy.get_place_context(text=sentence)
  #     nlp_wk = spacy.load('xx_ent_wiki_sm')
      
    adp =  [word for word,pos in phrase_sent if pos == 'ADP']
    prop = [word for word,pos in phrase_sent if pos == 'PROPN']
    city = [(X.text) for X in doc.ents]
    # print(adp)
    if(len(adp)>1):
      adp = str(adp)
      # print(city)
      adp = adp.replace("pour,", "")
      adp = adp.replace("en,", "")
      adp = adp.replace("en", "")
      adp = adp.replace("dans,", "")
      adp = adp.replace(" ", "")
      
      # print(adp)
      adp = nlp(adp)
      
      adp_word =  [(X, X.pos_) for X in adp]
      # print(adp_word)
      real_adp = [word for word,pos in adp_word if pos == 'ADP'] 
      adp = real_adp
      # print(len(str(adp)))
    dictionary = {"departure":["de","depuis","à", "par","vers"]}
    # print(prop)
    for cle, valeur in dictionary.items():
      if not adp or valeur[0] == str(adp[0]):
        if len(city) == 1:
          city = str(city)
          city = nlp(city)          
          city_word = [(X, X.pos_) for X in city]
          real_city = [word for word,pos in city_word if pos == 'PROPN'] 
          print(real_city)
          city = real_city
        if(str(city[0]) == ""):
          print("departure : ",prop[0])
        else:
          print("departure : ",city[0])
      elif valeur[4] == str(adp[0]):
        if(str(city[0]) == ""):
          print("departure : ",prop[0])
        else:
          print("departure : ",city[0])
      elif valeur[2] == str(adp[0]) and str(verb[1]) == "être":
        if(str(city[0]) == ""):
          print("departure : ",prop[0])
        else:
          print("departure : ",city[0])
      elif valeur[2] == str(adp[0]):
        if(str(city[1]) == ""):
          print("departure : ",prop[1])
        else:
          print("departure : ",city[1])
      elif valeur[2] == str(adp[0]):
        if(str(city[1]) == ""):
          print("departure : ",prop[1])
        else:
          print("departure : ",city[1])
      elif valeur[1] == str(adp[0]) or str(adp[1]):
        if(str(city[1]) == ""):
          print("departure : ",prop[1])
        else:
          print("departure : ",city[1])
      elif valeur[3] == str(adp[1]):
        if(str(city[1]) == ""):
          print("departure : ",prop[1])
        else:
          print("departure : ",city[1])
      else:
        if(str(city[0]) == ""):
          print("departure : ",prop[0])
        else:
          print("departure : ",city[0])
return_departure("Nice Lyon")

[Lyon]
departure :  Lyon


In [ ]:
def return_POS(sentence):
  if(sentence == ""):
    print("veuillez saisir votre trajet")
  else:
    return_departure(sentence)
    return_arrival(sentence)
return_POS("je veux aller à Sète et arriver à Strasbourg")

departure :  Strasbourg
arrival :  Sète


In [ ]:
def main_function(input):
    # => Transform input to text
    text = input
    if (check_if_text_accepted_language(text)):
        return_POS(text)
    else:
        print("/!\ Please, speak french")

In [ ]:
main_function("Je cherche un train pour aller de Paris à Carcassone")
print("-------------")

departure :  Paris
arrival :  Carcassone
-------------
